In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit, least_squares
import numpy as np
import math, random
import matplotlib.pyplot as plt
import nibabel as nib
import os
%matplotlib inline
np.random.seed(0)

In [ ]:
# 3 parameter objective function.

def objective (t,tmax, ymax,b):

    return ymax*((t/tmax)**((b*tmax)))*np.exp((tmax-t)*b)

In [ ]:
# 4 parameter objective function.

def objective2(t,tmax, ymax, b, offset):
    offInt = math.ceil(offset)
    offArr = np.zeros(offInt).astype(dtype=np.float32)
    offArr = np.append(offArr,ymax*((t[:(len(t)-offInt)]/tmax)**((b*tmax)))*np.exp((tmax-t[:(len(t)-offInt)])*b)).astype(np.float32)
    return offArr

In [ ]:
def preprocess_data (fmri_file):
    
    img = nib.load(fmri_file)
    hdr = img.header
    img_data = img.get_fdata()
    a, b, c, d = img_data.shape
    product = a*b*c
    arr = np.reshape(img_data, (product,d))
    new_arr = []
    for i in range(5):
        new_arr= np.where(arr[:,i] != 0)

    nonzero_time = arr[new_arr,:]

    arr_nonzero_time = nonzero_time[0][:][:]

    return arr, arr_nonzero_time, a, b, c, d, nonzero_time, new_arr, hdr

In [ ]:
def get_mat(arr_nonzero_time, d):
    size = len(arr_nonzero_time) 
    t = np.arange(1, d+1, 1)
    mat = np.array([t]*size)
    return mat

In [ ]:
def get_meann(arr_nonzero_time):
    meann = []
    for i in range(len(arr_nonzero_time)):
        meann.append(np.mean(arr_nonzero_time[i,1:30]))
        arr_nonzero_time[i,:] -= np.mean(arr_nonzero_time[i,1:30])
    return meann

In [ ]:
# Using modified scipy.

from myscipy.scipy import optimize

def step1(objective2, mat, arr_nonzero_time):

    result2 = []
    for i in range((len(arr_nonzero_time))):
        popt, _ = optimize.curve_fit(objective2, mat[i], arr_nonzero_time[i],p0=[231,5,0.005,30],bounds = ((0,0,0,20),(450,1000,0.1,200)), maxfev=len(arr_nonzero_time))
#         if i%1000 == 0:
#             print(i,"iterations done")
        result2.append(popt)
    return result2


In [ ]:
def get_initial_fits(result2):
    fits = []
    for i in range(len(result2)):
        fit = objective2(mat[i],*result2[i])
        fits.append(fit)
    return fits

In [ ]:
def get_initial_offsets(fits):
    offsets = [0]*len(fits)
    for i in range(len(fits)):
        func = fits[i]
        func = list(func)
        for j in range(len(func)):
            if func[j] > 0:
                offsets[i] = func.index(func[j])
                break
    return offsets

In [ ]:
def adjust_waveforms(arr_nonzero_time, offsets):
    arr_nonzero_time2 = [0]*len(arr_nonzero_time)
    for i in range(len(arr_nonzero_time)):
        offset = np.zeros(offsets[i])
        arr_nonzero_time2[i] = np.hstack((arr_nonzero_time[i,:(len(arr_nonzero_time[i])-offsets[i])], offset))
    return arr_nonzero_time2

In [ ]:
def create_new_mat(arr_nonzero_time, offsets):
    size = len(arr_nonzero_time)
    t = np.arange(offsets[i], d+1, 1)
    mat_2 = np.array([t]*size)
    return mat_2 

In [ ]:
from scipy.optimize import curve_fit as cf

def step2(objective, mat, arr_nonzero_time2):
    result1 = []
    for i in range((len(arr_nonzero_time))):
        popt, _ = cf(objective, mat[i], arr_nonzero_time2[i],p0=[231,5,0.005],bounds = ((0,0,0),(450,1000,0.1)), maxfev=len(arr_nonzero_time))
#         if i%1000 == 0:
#             print(i,"iterations done")
        result1.append(popt)
    return result1

In [ ]:
def get_final_fits(result1, result2, objective):
    newFits = []
    for i in range(len(result2)):
        tmax,ymax,b= result1[i]
        fit = objective(mat[i],tmax,ymax,b)
        newFits.append(fit)
    return newFits

In [ ]:
def get_adjusted_offsets(newFits):
    offsets = [0]*len(fits)
    for i in range(len(fits)):
        func = newFits[i]
        func = list(func)
        for j in range(len(func)):
            if func[j] > 10:
                offsets[i] = func.index(func[j])
                break
    return offsets

In [ ]:
def calculate_residuals(newFits, arr_nonzero_time):
    SSE = []
    for i in range(len(arr_nonzero_time)):
        loss = (newFits[i]-arr_nonzero_time[i])**2
        SSE.append(loss)
    return SSE

In [ ]:
def calculate_mean_residual(residuals, arr_nonzero_time):
    mean_residuals = []
    for i in range(len(arr_nonzero_time)):
        mean_residual = mean(residuals[i])
        mean_residuals.append(mean_residual)
    return mean_residuals

In [ ]:
def calculate_percentchange(ymax_list, meann):
    percent_change = []
    for i in range(len(ymax_list)):
        change = ymax_list[i]/meann[i]
        percent_change.append(change)
    return percent_change
        

In [ ]:
def recreate_original_arrays(result1, offsets, arr, a, b, c, d, meann, newFits, residuals, nonzero_time, new_arr):
    nonzero_time_copy = nonzero_time.copy()
    tmaxx, ymaxx, bb = zip(*result1)
    tmax_list = list(tmaxx)
    ymax_list = list(ymaxx)
    b_list = list(bb)

    tmaxArr = np.zeros(len(arr))
    ymaxArr = np.zeros(len(arr))
    bArr = np.zeros(len(arr))

    tmaxArr[new_arr] = tmax_list
    ymaxArr[new_arr] = ymax_list
    bArr[new_arr] = bb
    
    offsetArr = np.zeros(len(arr))
    offsetArr[new_arr] = offsets
    
    meann = np.array(meann)
    meanArr = np.zeros(len(arr))
    meanArr[new_arr] = meann
    
    newFits = np.array(newFits).astype('int')
    fitsArr = np.zeros(arr.shape)
    nonzero_time[0][:][:] = newFits 
    fitsArr[new_arr,:]= nonzero_time 
    
    residualsArr = np.zeros(arr.shape)
    nonzero_time_copy[0][:][:] = residuals
    residualsArr[new_arr,:] = nonzero_time_copy
    
    mean_residualArr = np.zeros(arr.shape)
    mean_residuals = np.array(mean_residuals)
    mean_residualArr[new_arr] = mean_residuals
    
    percentChangeArr = np.zeros(arr.shape)
    percent_change = np.array(percent_change)
    percentChangeArr[new_arr] = percent_change
    
    
    meanFits = np.reshape(meanArr,(a,b,c))
    tmaxFits = np.reshape(tmaxArr, (a,b,c))
    ymaxFits = np.reshape(ymaxArr, (a,b,c))
    bFits = np.reshape(bArr,(a,b,c))
    offsetFits = np.reshape(offsetArr,(a,b,c))
    newfits = np.reshape(fitsArr,(a,b,c,d))
    residuals = np.reshape(residualsArr,(a,b,c,d))
    
    return meanFits, tmaxFits, ymaxFits, bFits, offsetFits, newfits, residuals, ymax_list

In [ ]:
def recreate_og_arrays(arr, a, b, c, d, mean_residuals, percent_change):
    mean_residualArr = np.zeros(arr.shape)
    mean_residuals = np.array(mean_residuals)
    mean_residualArr[new_arr] = mean_residuals
    
    percentChangeArr = np.zeros(arr.shape)
    percent_change = np.array(percent_change)
    percentChangeArr[new_arr] = percent_change
    
    mean_residual = np.reshape(mean_residualArr,(a,b,c))
    percentChange = np.reshape(percentChangeArr,(a,b,c))
    
    return mean_residual, percentChange

In [ ]:
def save_as_nifti_files(meanFits, tmaxFits, ymaxFits, bFits, offsetFits, newfits, residuals, name, hdr, mean_residual, percentChange):
    path2 = '/home/arya/Downloads/newData/*/*/'
    folder_name = "fits_"+name
    folder = os.path.join(path2, folder_name,"")
    os.makedirs(folder)
    
    nifti_file_mean = nib.Nifti1Image(meanFits, None, header = hdr)
    nib.save(nifti_file_mean, os.path.join(folder, "mean.nii.gz"))


    nifti_file_tmax = nib.Nifti1Image(tmaxFits, None, header = hdr)
    nib.save(nifti_file_tmax, os.path.join(folder, "tmax.nii.gz"))

    nifti_file_ymax = nib.Nifti1Image(ymaxFits, None, header = hdr)
    nib.save(nifti_file_ymax, os.path.join(folder,"ymax.nii.gz"))
    
    nifti_file_b = nib.Nifti1Image(bFits, None, header = hdr)
    nib.save(nifti_file_b, os.path.join(folder, 'shape.nii.gz'))

    nifti_file_offset = nib.Nifti1Image(offsetFits, None, header = hdr)
    nib.save(nifti_file_offset, os.path.join(folder, 'offsets.nii.gz'))

    nifti_file_fits = nib.Nifti1Image(newfits, None, header = hdr)
    nib.save(nifti_file_fits, os.path.join(folder, "fits.nii.gz"))
    
    nifti_file_residuals = nib.Nifti1Image(residuals, None, header = hdr)
    nib.save(nifti_file_residuals, os.path.join(folder, "residuals.nii.gz"))
    
    nifti_file_mean_residual = nib.Nifti1Image(mean_residual, None, header = hdr)
    nib.save(nifti_file_mean_residual, os.path.join(folder, "mean_residual.nii.gz"))
    
    nifti_file_percent_change = nib.Nifti1Image(percentChange, None, header = hdr)
    nib.save(nifti_file_percent_change, os.path.join(folder, "percentchange.nii.gz"))

In [ ]:
path = '/home/arya/Downloads/newData/*/*/*/filtered*'

import glob
files = []
for f in glob.glob(path):
    files.append(f)
        
for f in files:
    name = f.split('/')[5]
    arr, arr_nonzero_time, a, b, c, d, nonzero_time, new_arr, hdr = preprocess_data(f)
    print("Pre-processing done")
    mat = get_mat(arr_nonzero_time, d)
    print("Created mat")
    meann = get_meann(arr_nonzero_time)
    print("Got the mean")
    result2 = step1(objective2, mat, arr_nonzero_time)
    print("Step 1 done")
    fits = get_initial_fits(result2)
    print("Initial fits estimated")
    offsets = get_initial_offsets(fits)
    print("offsets calculated")
    arr_nonzero_time2 = adjust_waveforms(arr_nonzero_time, offsets)
    print("Waves adjusted")
    result1 = step2(objective, mat, arr_nonzero_time2)
    print("step2 done")
    newFits = get_final_fits(result1, result2, objective)
    print("Final fits estimated")
    newoffsets = get_adjusted_offsets(newFits)
    print("adjusted offsets calculated")
    residuals = calculate_residuals(newFits, arr_nonzero_time)
    print("residuals calculation done")
    meanFits, tmaxFits, ymaxFits, bFits, offsetFits, newfits, residualfits, ymax_list = recreate_original_arrays(result1, offsets, arr, a, b, c, d, meann, newFits, residuals, nonzero_time, new_arr)
    print("Arrays created")
    mean_residuals = calculate_mean_residual(residuals, arr_nonzero_time)
    percent_change = calculate_percentchange(ymax_list, meann)
    mean_residual, percentChange = recreate_og_arrays(arr, a, b, c, d, mean_residuals, percent_change)
    save_as_nifti_files(meanFits, tmaxFits, ymaxFits, bFits, offsetFits, newfits, residualfits, name, hdr, mean_residual, percentChange)
    print("dataset_"+ name+ " done")